In [1]:
!rm -r AI2020/
!git clone https://github.com/UmbertoJr/AI2020.git &> /dev/null

In [2]:
!ls AI2020/images/

tic-tac-toe.png


# AI search for the Tic-tac-toe

## The *Game*

In [120]:
import time
import numpy as np

class Game:
  def __init__(self):
    self.current_state = np.array([['.','.','.'],
                                   ['.','.','.'],
                                   ['.','.','.']])
    # Player X always plays first
    self.player_turn = 'X'

  def draw_board(self):
    for i in range(0, 3):
      for j in range(0, 3):
        print('{}|'.format(self.current_state[i][j]), end=" ")
      print()

  # Determines if the made move is a legal move
  def is_valid(self, px, py):
    if px < 0 or px > 2 or py < 0 or py > 2:
      return False
    elif self.current_state[px][py] != '.':
      return False
    else:
      return True

  # Checks if the game has ended and returns the winner in each case
  def is_end(self):
    # Vertical win
    for i in range(0, 3):
      if (self.current_state[0][i] != '.' and
          self.current_state[0][i] == self.current_state[1][i] and
          self.current_state[1][i] == self.current_state[2][i]):
        return self.current_state[0][i]

    # Horizontal win
    for i in range(0, 3):
      if (self.current_state[i].tolist() == ['X', 'X', 'X']):
        return 'X'
      elif (self.current_state[i].tolist() == ['O', 'O', 'O']):
        return 'O'

    # Main diagonal win
    if (self.current_state[0][0] != '.' and
        self.current_state[0][0] == self.current_state[1][1] and
        self.current_state[0][0] == self.current_state[2][2]):
      return self.current_state[0][0]

    # Second diagonal win
    if (self.current_state[0][2] != '.' and
        self.current_state[0][2] == self.current_state[1][1] and
        self.current_state[0][2] == self.current_state[2][0]):
      return self.current_state[0][2]

    # Is whole board full?
    for i in range(0, 3):
        for j in range(0, 3):
            # There's an empty field, we continue the game
            if (self.current_state[i][j] == '.'):
                return None

    # It's a tie!
    return '.'


  def state(self):
    self.winner = self.is_end()
    if self.winner in ["X","O","."]:
      return 'game over'
    return 'continue'

  def static_eval(self):
    if self.winner == "X":
      return 1
    elif self.winner == "O":
      return -1
    else:
      return 0


  def children(self):
    if self.player_turn == 'X':
      for i in range(3):
        for j in range(3):
          if self.is_valid(i, j):
            child = Game()
            child.current_state = np.copy(self.current_state)
            child.current_state[i][j] = "X"
            child.player_turn = "O"
            yield child

    elif self.player_turn == 'O':
      for i in range(3):
        for j in range(3):
          if self.is_valid(i, j):
            child = Game()
            child.current_state = np.copy(self.current_state)
            child.current_state[i][j] = "O"
            child.player_turn = "X"
            yield child


def play(game, strategy, alphabeta=False, to_plot=True):
  remaining_pos = 9
  while True:
    alpha, beta = -2, 2
    result = game.is_end()
    if to_plot:
      game.draw_board()
      print()

    # Printing the appropriate message if the game has ended
    if result != None:
      if to_plot:  
        if result == 'X':
          print('The winner is X!')
        elif result == 'O':
          print('The winner is O!')
        elif result == '.':
          print("It's a tie!")
      return

    qx, qy = None, None
    if game.player_turn == 'X':
      max_move = -2
      start = time.time()
      for i in range(3):
        for j in range(3):
          if beta <= alpha:
                break
          if game.is_valid(i, j):
            game.current_state[i][j] = "X"
            game.player_turn = "O"
            state_value = strategy(game, remaining_pos, True, alpha, beta )
            game.current_state[i][j] = "."
            game.player_turn = "X"
            if state_value > max_move:
              m, qx, qy = state_value, i, j
              max_move = m
            if alphabeta:
              alpha = max(alpha, state_value)

        if beta <= alpha:
          break

      end = time.time()
      
      if to_plot:  
        print('Evaluation time: {}s'.format(round(end - start, 7)))
        print('move X: X = {}, Y = {}'.format(qx, qy))
        if alphabeta:
          print(f'alpha {alpha}, beta {beta}')
      game.current_state[qx][qy] = 'X'
      game.player_turn = 'O'
      remaining_pos -= 1

    else:
      min_move = 2
      start = time.time()
      for i in range(3):
        for j in range(3):
          if beta <= alpha:
                break
          if game.is_valid(i, j):
            game.current_state[i][j] = "O"
            game.player_turn = "X"
            state_value = strategy(game, remaining_pos, False, alpha, beta )
            game.current_state[i][j] = "."
            game.player_turn = "O"
            if state_value < min_move:
              m, qx, qy = state_value, i, j
              min_move = m
            if alphabeta:
              beta = min(beta, state_value)
              
        if beta <= alpha:
          break
              
      end = time.time()
      if to_plot:  
        print('Evaluation time: {}s'.format(round(end - start, 7)))
        print('move O: X = {}, Y = {}'.format(qx, qy))
        if alphabeta:
          print(f'alpha {alpha}, beta {beta}')
      game.current_state[qx][qy] = 'O'
      game.player_turn = 'X'
      remaining_pos -= 1


## Min Max

In [111]:
def minimax(game, depth, maximizingPlayer, *args):
  if depth == 0 or 'game over' in game.state():
    return game.static_eval()

  if maximizingPlayer:
    maxEval = -2
    for child in game.children():
      eval_position = minimax(child, depth - 1, maximizingPlayer=False) 
      maxEval = max(maxEval, eval_position)
    return maxEval

  else:
    minEval = 2
    for child in game.children():
      eval_position = minimax(child, depth - 1, maximizingPlayer=True) 
      minEval = min(minEval, eval_position)
    return minEval
  

In [112]:
g = Game()
play(g, minimax)

.| .| .| 
.| .| .| 
.| .| .| 

Evaluation time: 13.7935929s
move X: X = 0, Y = 0
X| .| .| 
.| .| .| 
.| .| .| 

Evaluation time: 2.6524286s
move O: X = 0, Y = 2
X| .| O| 
.| .| .| 
.| .| .| 

Evaluation time: 0.3783245s
move X: X = 1, Y = 0
X| .| O| 
X| .| .| 
.| .| .| 

Evaluation time: 0.0400286s
move O: X = 2, Y = 0
X| .| O| 
X| .| .| 
O| .| .| 

Evaluation time: 0.008553s
move X: X = 1, Y = 1
X| .| O| 
X| X| .| 
O| .| .| 

Evaluation time: 0.001683s
move O: X = 2, Y = 2
X| .| O| 
X| X| .| 
O| .| O| 

Evaluation time: 0.0003426s
move X: X = 1, Y = 2
X| .| O| 
X| X| X| 
O| .| O| 

The winner is X!


In [125]:

list_time = []
for _ in range(100):
  g = Game()
  start = time.time()
  play(g, minimax, to_plot=False)
  list_time.append(time.time() - start)

print(f"average time for minimax: {np.mean(list_time)} +- {np.std(list_time)}")

average time for minimax: 15.11209282398224 +- 0.1813645609383075


# Alpha Beta pruning

In [123]:
def minimax_pruned(game, depth, maximizingPlayer, alpha, beta):
  if depth == 0 or 'game over' in game.state():
    return game.static_eval()

  if maximizingPlayer:
    maxEval = -2
    for child in game.children():
      eval_position = minimax_pruned(child, depth - 1, False, alpha, beta) 
      maxEval = max(maxEval, eval_position)
      alpha = max(alpha, eval_position)
      if beta <= alpha:
        break
    return maxEval

  else:
    minEval = 2
    for child in game.children():
      eval_position = minimax_pruned(child, depth - 1, True, alpha, beta) 
      minEval = min(minEval, eval_position)
      beta = min(beta, eval_position)
      if beta <= alpha:
        break
    return minEval

In [124]:
# test on problem
g = Game()
play(g, minimax_pruned, alphabeta=True)

.| .| .| 
.| .| .| 
.| .| .| 

Evaluation time: 0.2931805s
move X: X = 0, Y = 0
alpha 1, beta 2
X| .| .| 
.| .| .| 
.| .| .| 

Evaluation time: 0.2008045s
move O: X = 0, Y = 2
alpha -2, beta 0
X| .| O| 
.| .| .| 
.| .| .| 

Evaluation time: 0.073489s
move X: X = 1, Y = 0
alpha 1, beta 2
X| .| O| 
X| .| .| 
.| .| .| 

Evaluation time: 0.0194461s
move O: X = 2, Y = 0
alpha -2, beta -1
X| .| O| 
X| .| .| 
O| .| .| 

Evaluation time: 0.0057344s
move X: X = 1, Y = 1
alpha 1, beta 2
X| .| O| 
X| X| .| 
O| .| .| 

Evaluation time: 0.0038157s
move O: X = 2, Y = 2
alpha -2, beta -1
X| .| O| 
X| X| .| 
O| .| O| 

Evaluation time: 0.0003941s
move X: X = 1, Y = 2
alpha 1, beta 2
X| .| O| 
X| X| X| 
O| .| O| 

The winner is X!


In [126]:

list_time = []
for _ in range(1000):
  g = Game()
  start = time.time()
  play(g, minimax_pruned, alphabeta=True, to_plot=False)
  list_time.append(time.time() - start)

print(f"average time for minimax with alpha-beta: {np.mean(list_time)} +- {np.std(list_time)}")

average time for minimax with alpha-beta: 0.2921559052467346 +- 0.019459386393990433
